# Retrieve a wandb artifact of results and process

In [1]:
import sys
sys.path.append('..')

import wandb
import yaml
import pickle
import tempfile
from tqdm import tqdm
import numpy as np
import pandas as pd
import os

from src.utils2 import process_results

with open('../wandb.yaml') as file:
    wandb_data = yaml.load(file, Loader=yaml.FullLoader)

os.environ["WANDB_API_KEY"] = wandb_data['wandb_key'] 
wandb_entity = str(wandb_data['wandb_entity'])

# your wandb project name
project_name = 'dips_data_efficiency'

In [2]:
def process(overall_result_dicts, numIters, subsample_res, prop_check, method='pseudo'):
    results = process_results(results_list=overall_result_dicts,numIters=numIters, end_score=True)
    sample_processed = {}

    for subsample_prop in prop_check:
        sample_processed[subsample_prop] = process_results(results_list=subsample_res[subsample_prop],numIters=numIters, end_score=True)


    our_scores= []
    pseudo_score = []
    final_props = []
    for subsample_prop in prop_check:
        try:
            score = sample_processed[subsample_prop][method]['dips_full_mean']
            our_scores.append(score)

            score = sample_processed[subsample_prop][method]['vanilla_mean']
            pseudo_score.append(score)

            final_props.append(subsample_prop)
        except:
            pass


    score = results[method]['dips_full_mean']
    our_scores.append(score)

    score = results[method]['vanilla_mean']
    pseudo_score.append(score)
 
    return pseudo_score, our_scores, final_props

In [3]:
# minimum sweeps
sweeps = [('compas', '1.0'), ('covid', '0.66'),  ('adult', '0.66'), ('maggic', '0.66')]

# all sweeps
# sweeps = [('seer', '1.0'), ('adult', '0.66'), ('cutract', '1.0'), ('covid', '0.66'),   ('maggic', '0.66'), ('German-credit', '1.0'),("compas", "1.0"), ("agaricus-lepiota", '1.0'),  ("higgs", "0.1"),   ('drug', '0.1'), ("blog", "0.2"), ("credit", "1.0")]

# either 'ups' or 'pseudo'
method = 'ups'

selector = 'dips'

ours_all_data = []
vanilla_all_data = []

for idx_sweep, sweep in tqdm(enumerate(sweeps)):
    try:
        files=0
        prop_data = sweep[1]
        dataset_name = sweep[0]
        numTrials = 5
        seed = 0
        data_iq_xthresh = "0.0"

        results_dfs = []
        metainfo_list = []
        min_list = []
        max_list = []

        our_scores_all = []
        pseudo_score_all = []
        our_scores_se_all = []
        pseudo_score_se_all = []
        

        for idx, seed in enumerate([0, 10, 42, 100, 1000]):
            try:
            
                temp_dir = tempfile.TemporaryDirectory()

                metainfo = f"{dataset_name}_{prop_data}_{data_iq_xthresh}_{numTrials}_{seed}"
                metainfo_list.append(metainfo)

                api = wandb.Api()
                artifact = api.artifact(f'{wandb_entity}/{project_name}/sample_dict_{metainfo}:latest', type='pickle')
                artifact_dir = artifact.download(root=temp_dir.name)


                with open(artifact_dir + f"/sample_dict_{metainfo}.pkl", "rb") as f:
                    subsample_res = pickle.load(f)
                temp_dir.cleanup()


                temp_dir = tempfile.TemporaryDirectory()


                api = wandb.Api()
                artifact = api.artifact(f'{wandb_entity}/{project_name}/results_dict_{metainfo}:latest', type='pickle')
                artifact_dir = artifact.download(root=temp_dir.name)


                with open(artifact_dir + f"/results_dict_{metainfo}.pkl", "rb") as f:
                    overall_result_dicts = pickle.load(f)
                temp_dir.cleanup()

                prop_check = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
                numIters = 5
                pseudo_score, our_scores,  final_props = process(overall_result_dicts, numIters, subsample_res, prop_check, method)
                
                our_scores_all.append(our_scores)
                pseudo_score_all.append(pseudo_score)

                print(f"Loaded {dataset_name}, {files}")

                files+=1


            except Exception as e:
                continue


        our_scores_all = np.array(our_scores_all)
        pseudo_score_all = np.array(pseudo_score_all)

        from scipy.stats import sem
        our_scores = np.mean(our_scores_all, axis=0)
        pseudo_score = np.mean(pseudo_score_all, axis=0)

        try:
            our_scores_se = sem(our_scores_all, axis=0)
            pseudo_score_se = sem(pseudo_score_all, axis=0)
        except:
            our_scores_se = np.std(our_scores_all, axis=0)
            pseudo_score_se = np.std(pseudo_score_all, axis=0)
        
        ours_all_data.append(our_scores)
        vanilla_all_data.append(pseudo_score)
    except Exception as e:
        print(e)
        continue


0it [00:00, ?it/s]

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


Loaded compas, 0


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


Loaded compas, 1


...

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


Loaded maggic, 2


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


Loaded maggic, 3


4it [00:34,  8.53s/it]


In [4]:
from copy import deepcopy
# compute the norm vs the highest vanilla method value (i.e. relative gain)
norms = np.array(vanilla_all_data)[:,-1]
ours_gain = deepcopy(np.array(ours_all_data))
vanilla_gain = deepcopy(np.array(vanilla_all_data))

for i in range(vanilla_gain.shape[0]):
    vanilla_gain[i,:] = vanilla_gain[i,:]-norms[i]
    ours_gain[i, :] = ours_gain[i,:]-norms[i]

In [5]:
# pickle to plot later

import pickle

with open(f"../results/efficiency_norms_{selector}_{method}.pkl", "wb") as f:
    pickle.dump(norms, f)

with open(f"../results//vanilla_efficiency_{selector}_{method}.pkl", 'wb') as file:
    pickle.dump(vanilla_gain, file)

with open(f"../results/ours_efficiency_{selector}_{method}.pkl", 'wb') as file:
    pickle.dump(ours_gain, file)